In [1]:
import os
import fastavro
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from correction.compute import *

In [2]:
from alerce.api import AlerceAPI
client = AlerceAPI()

In [3]:
AVRO_PATH = "../tests/data_examples/"

# Functions to manipulate example data

In [4]:
def read_avro(avro_path):
    with open(avro_path, "rb") as file:
        try:
            reader = fastavro.reader(file)
            data = reader.next()
        except Exception as e:
           return None
    return data

In [5]:
def get_avros(oid):
    path = os.path.join(AVRO_PATH, oid)
    avros = sorted(os.listdir(path))
    data = []
    for a in avros:
        content = read_avro(os.path.join(path, a))
        if content:
            data.append(content)
    return data

In [77]:
detection_columns = ['oid', 'candid', 'jd', 'fid', 'pid', 'diffmaglim', 'isdiffpos', 'nid', 'ra', 'dec', 'magpsf', 'sigmapsf', 'magap', 'sigmagap', 'distnr', 'rb', 'rbversion', 'drb', 'drbversion', 'magapbig', 'sigmagapbig', 'rfid', 'magpsf_corr', 'sigmapsf_corr', 'sigmapsf_corr_ref', 'has_stamp', 'corr', 'flag', 'candid_alert']
non_detection_columns = ['oid', 'jd', 'fid', 'diffmaglim']
object_columns = ['oid', 'ndethist', 'ncovhist', 'jdstarthist', 'jdendhist', 'stellar', 'ndet', 'meanra', 'meandec', 'sigmara', 'sigmadec', 'deltajd', 'firstmjd', 'lastmjd']
magstats_columns = ['oid', 'rfid', 'fid', 'magnr', 'nrfid', 'stellar', 'corr', 'ndet', 'dmdt', 'magmean', 'magmedian', 'magmax', 'magmin', 'magsigma', 'maglast', 'magfirst', 'firstmjd', 'lastmjd', 'step_id_corr', 'step_id_feat']


In [147]:
# This function must be implement in step, but querying to db
def is_first_detection(df, alert):
    candid = alert["candid"]
    oid = alert["objectId"]
    fid = alert["candidate"]["fid"]
    mask = (df["oid"] == oid) & (df["fid"] == fid)
    table = df[mask]
    return len(table) == 0

def get_first_magnr(df, data):
    oid = data["objectId"]
    candid = data["candidate"]["fid"]
    mask = (df["oid"] == oid) & (df["fid"] == fid)
    return df[mask]["magnr"].values[0]

def insert_detection(df, alert, magpsf_corr, sigmapsf_corr, sigmapsf_corr_ref, flag, has_stamp, corr):
    data = {}
    for k in alert["candidate"]:
        if k in detection_columns:
            data.update({k: alert["candidate"][k]})
    data.update({
        "oid": alert["objectId"],
        "magpsf_corr": magpsf_corr, 
        "sigmapsf_corr": sigmapsf_corr, 
        "sigmapsf_corr_ref": sigmapsf_corr_ref, 
        "flag": flag, 
        "has_stamp": has_stamp, 
        "corr": corr
    })
    return df.append([data])


def insert_non_detection(df, oid, jd, fid, diffmaglim):
    data = {
        "oid": oid,
        "jd": jd,
        "fid": fid,
        "diffmaglim": diffmaglim
    }
    return df.append([data])

def insert_object(df, alert, stellar):
    data = {}
    for k in alert["candidate"]:
        if k in object_columns:
            data.update({k: alert["candidate"][k]})
    data.update({
        "oid": alert["objectId"],
        "stellar": stellar
    })
    return df.append([data])

def insert_magstats(df, alert):
    data = {}
    for k in alert["candidate"]:
        if k in magstats_columns:
            data.update({k: alert["candidate"][k]})
    data.update({
        "oid": alert["objectId"],
    })
    return df.append([data])

In [148]:
oids = os.listdir(AVRO_PATH)

In [149]:
detection_table = pd.DataFrame(columns=detection_columns)
non_detection_table = pd.DataFrame(columns=non_detection_columns)
object_table = pd.DataFrame(columns=object_columns)
magstats_table = pd.DataFrame(columns=magstats_columns)

In [150]:
for idx, data in enumerate(example):
    is_first = is_first_detection(detection_table, data)
    has_stamp = True
    fid = data["candidate"]["fid"]    
    candid =  data["candid"]
    
    if is_first:
        magstats_table = insert_magstats(magstats_table, data)
    
    # object
    stellar_object, stellar_magstats = validate_object(data["candidate"], is_first)
    corr_detection, corr_magstats, flag = validate_magnitudes(data["candidate"])
    magpsf_corr, sigmapsf_corr, sigmapsf_corr_ref = apply_correction(data["candidate"])
    
    detection_table = insert_detection(detection_table, data,  magpsf_corr, sigmapsf_corr, sigmapsf_corr_ref, flag, has_stamp, corr_detection)

display(df)

first
not first
first
not first
not first
not first
not first
not first
not first
not first
not first
not first


""


In [151]:
magstats_table

,oid,rfid,fid,magnr,nrfid,stellar,corr,ndet,dmdt,magmean,magmedian,magmax,magmin,magsigma,maglast,magfirst,firstmjd,lastmjd,step_id_corr,step_id_feat
0,ZTF19abaejrh,599120201,2,18.254000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ZTF19abaejrh,599120101,1,18.702999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
detection_table

,oid,candid,jd,fid,pid,diffmaglim,isdiffpos,nid,ra,dec,...,magapbig,sigmagapbig,rfid,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ref,has_stamp,corr,flag,candid_alert
0,ZTF19abaejrh,899443400115015003,2.458654e+06,2,899443400115,19.675455,t,899,356.892535,16.350683,...,19.393801,0.4283,599120201,18.021721,NaN,0.033414,True,True,False,NaN
0,ZTF19abaejrh,903437770115015004,2.458658e+06,2,903437770115,19.952297,t,903,356.892514,16.350608,...,18.878201,0.2005,599120201,17.873658,NaN,0.039245,True,True,False,NaN
0,ZTF19abaejrh,909460460115015007,2.458664e+06,1,909460460115,20.321400,t,909,356.892516,16.350709,...,18.831800,0.1273,599120101,18.030626,NaN,0.042893,True,True,False,NaN
0,ZTF19abaejrh,912422990115015012,2.458667e+06,2,912422990115,20.625626,t,912,356.892525,16.350711,...,18.867500,0.1159,599120201,17.806076,NaN,0.031710,True,True,False,NaN
0,ZTF19abaejrh,915419730115015005,2.458670e+06,2,915419730115,20.523502,t,915,356.892498,16.350716,...,19.121099,0.1575,599120201,17.804514,NaN,0.030875,True,True,False,NaN
0,ZTF19abaejrh,915461510115015011,2.458670e+06,1,915461510115,20.775969,t,915,356.892515,16.350701,...,19.042000,0.1071,599120101,18.132973,NaN,0.049120,True,True,False,NaN
0,ZTF19abaejrh,918427310115015009,2.458673e+06,1,918427310115,20.743717,t,918,356.892521,16.350690,...,19.205000,0.1241,599120101,18.168427,NaN,0.041817,True,True,False,NaN
0,ZTF19abaejrh,918463460115015018,2.458673e+06,2,918463460115,20.536043,t,918,356.892484,16.350728,...,19.206200,0.1571,599120201,17.868206,NaN,0.026872,True,True,False,NaN
0,ZTF19abaejrh,921447740115015009,2.458676e+06,1,921447740115,20.755289,t,921,356.892485,16.350711,...,19.744400,0.2111,599120101,18.272505,NaN,0.035381,True,True,False,NaN
0,ZTF19abaejrh,924477220115015006,2.458679e+06,2,924477220115,20.695583,t,924,356.892511,16.350677,...,19.156799,0.1476,599120201,17.954936,NaN,0.022873,True,True,False,NaN


In [153]:
detection_table.columns

Index(['oid', 'candid', 'jd', 'fid', 'pid', 'diffmaglim', 'isdiffpos', 'nid',
       'ra', 'dec', 'magpsf', 'sigmapsf', 'magap', 'sigmagap', 'distnr', 'rb',
       'rbversion', 'drb', 'drbversion', 'magapbig', 'sigmagapbig', 'rfid',
       'magpsf_corr', 'sigmapsf_corr', 'sigmapsf_corr_ref', 'has_stamp',
       'corr', 'flag', 'candid_alert'],
      dtype='object')